<a href="https://colab.research.google.com/github/MinsuChoKW/Data-Analytics-2025/blob/main/week7_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Week 7 - 예측 프로세스 모니터링 (Predictive Process Monitoring) (PM4Py)
- 실습 항목: Prefix Encoding, Prediction 등

## 환경 설정 (Setup)

본 노트북에서는 아래의 라이브러리를 활용한다:

* [PM4Py](https://pm4py.fit.fraunhofer.de/)
* [pandas](https://pandas.pydata.org/)
* [PyTorch](https://pytorch.org/)

In [ ]:
## Perform the commented out commands to install the dependencies
# %pip install pandas
# %pip install matplotlib
# %pip install pm4py
# %pip install torch
# %pip install tqdm

In [ ]:
import numpy as np
import pandas as pd
import pm4py
import os
import torch
import torch.nn as nn
from tqdm.autonotebook import tqdm

# Predictive Process Mining

## Event Log

본 예제에서는 `Sepsis` 로그를 활용한다.

In [ ]:
from pm4py.objects.log.importer.xes import importer as xes_importer

sepsis_log = xes_importer.apply('../data/sepsis.xes')

In [ ]:
len(sepsis_log)

## Prefix Extraction

이벤트 로그를 기반으로는 다양한 예측 작업을 수행할 수 있다. 일반적으로는 한 트레이스의 접두(prefix) 부분만 알려져 있고, 그 트레이스가 나타내는 프로세스 인스턴스의 미래 상태를 예측해야 한다는 가정을 둔다.

첫 단계는 이벤트 로그에 포함된 트레이스들로부터 적절한 prefix를 생성하여 이를 학습 샘플로 사용하는 것입니다. 간단한 예로, 우리는 환자가 프로세스 중에 최종 이벤트로 *Return ER (응급실 재방문)*이 발생하는지를 예측할 수 있다. 이 때 Return ER 이벤트는 이벤트 로그에 이미 포함되어 있기 때문에, 해당 이벤트를 제거하고 그것이 어떤 트레이스에서 발생했는지를 기억해 두어야 한다.

In [ ]:
sepsis_returns = [len(list(filter(lambda e: e["concept:name"] == "Return ER" ,trace))) > 0 for trace in sepsis_log]

In [ ]:
# check if this worked
print(sepsis_log[3][-1])
print(sepsis_returns[3])

print(sepsis_log[0][-1])
print(sepsis_returns[0])

동시에, 우리는 환자가 응급실을 재방문하는지, prefix의 길이에 따라 얼마나 잘 예측할 수 있는지에도 관심을 가질 수 있다. 예를 들어, 각 트레이스에서 최대 10개의 이벤트까지만 남겨 새로운 이벤트 로그를 생성할 수 있는데, 이를 10-prefix라고 한다.

**주의할 점은, 여기서 10이라는 값은 단순히 임의로 정한 길이일 뿐이며, 일반적으로는 특정 길이만이 아니라 다양한 길이나 모든 길이에 대한 접두를 생성한다는 것이다. 또한, 일부 트레이스는 10개 미만의 이벤트를 가지므로 이 경우 전체 트레이스를 접두로 사용하게 되는데, 이는 실제 예측 작업에서는 크게 유용하지 않을 수 있다.**

In [ ]:
# remove Return ER event
sepsis_log = pm4py.filter_event_attribute_values(sepsis_log, "concept:name", "Return ER", level = "event", retain=False)

from pm4py.objects.log.obj import EventLog, Trace
# generate prefixes, note that we need to add the casts to EventLog and Trace to make sure that the result is a PM4Py EventLog object
sepsis_prefixes = EventLog([Trace(trace[0:10], attributes = trace.attributes) for trace in sepsis_log])

In [ ]:
# check the trace length
print([len(trace) for trace in sepsis_log][0:15])
print([len(trace) for trace in sepsis_prefixes][0:15])

## Prefix Encoding

예측 모델을 학습하기 위해, 트레이스나 이벤트 시퀀스를 벡터 표현(vector representation) 으로 변환해야한다. 여기서는 PM4Py의 [feature selection and processing](https://pm4py.fit.fraunhofer.de/documentation#decision-trees) 기능을 이용하여 세 가지 기본적인 인코딩 방식을 계산하는 방법을 소개한다.

물론, 더 복잡한 인코딩도 가능하다. 예를 들어, 각 트레이스를 피처의 시퀀스로 표현하여 LSTM과 같은 순차 모델(sequential models)에서 사용할 수도 있다.

### Feature Selection \& Engineering

Prefix encoding을 수행하기 전에, 예측에 사용할 features를 선택해야 한다. 본 예제에서는 이벤트의 "activity" 속성만을 feature로 사용한다. 그러나 예측 문제에 따라 추가적인 트레이스/이벤트 속성을 포함시킬 수도 있다.

추가적으로, 새로운 트레이스 수준 feature(예: 요일, 케이스 시작 이후 경과 시간)이나 로그 기반 feature(예: 자원(resource)의 워크로드, 특정 시점에서의 활성 케이스 수) 등을 파생하여 활용할 수도 있다.

### Encoding as Set of Events

In [ ]:
from pm4py.algo.transformation.log_to_features import algorithm as log_to_features

# log_to_feature provides a flexible interface to compute features on an event and trace level
data, feature_names = log_to_features.apply(sepsis_prefixes, parameters={"str_ev_attr": ["concept:name"]})

`concept:name` 속성(즉, 이벤트 라벨)의 표준적인 인코딩 방식은 *원-핫 인코딩(one-hot encoding)* 벡터이다. 본 인코딩을 살펴보면, 각 숫자의 인덱스는 특징 라벨 벡터(feature label vector) 내의 인덱스와 정확히 대응된다.

즉, 이벤트 라벨 집합이 주어졌을 때, 특정 이벤트가 해당 집합의 어느 위치에 있는지를 기준으로 벡터가 구성되며, 그 위치에는 1이, 나머지 모든 위치에는 0이 들어간다.

In [ ]:
from pm4py.objects.log.util.log import project_traces
def project_nth(log, index):
    print(str(project_traces(log)[index]))

In [ ]:
project_nth(sepsis_prefixes, 0)

In [ ]:
print(feature_names)

In [ ]:
print(data[0])

구성된 데이터 형태는 다음과 같다.

In [ ]:
np.asarray(data).shape

즉, PM4Py는 이벤트 로그에 대한 소위 *집합 추상화(set abstraction)*의 *원-핫 인코딩(one-hot encoding)*을 제공한다. 이는 이벤트 로그 안에 총 16개의 구별되는 activity가 있으며, 피처 벡터는 단순히 해당 activity가 데이터에 존재하는지를 0과 1로 표현한다는 뜻이다.

이제 이러한 피처 벡터들이 어떤 분포(distribution)를 이루는지 살펴본다.

In [ ]:
# look at the unique vectors and their occurrence frequency/count
dist_features = np.unique(data, return_counts= True, axis = 0)
print(dist_features)

이 중 무엇이 가장 흔한 feature 벡터인가?

In [ ]:
# argmax give use the index of the most frequent vector
dist_features[0][np.argmax(dist_features[1])]

실제로 본 프로세스에서는 거의 모든 activity가 발생하여 선택지가 많지 않다. 즉, 이 인코딩 방식은 가장 유용한 방법이라고 보기는 어렵지만, 아주 단순한 방식이라 할 수 있다.

### Encoding as Bi-Grams / Succession Relation

In [ ]:
data_2gram, feature_names = log_to_features.apply(sepsis_prefixes,
                                                  parameters={"str_ev_attr": [],
                                                        "str_tr_attr": [],
                                                        "num_ev_attr": [],
                                                        "num_tr_attr": [],
                                                        "str_evsucc_attr": ["concept:name"]})
feature_names

각 feature는 이벤트 로그에서 임의의 두 activity 사이의 순차 관계(succession relation, 즉 bigram)를 나타낸다. 우리는 이러한 특징들을 텐서(tensor) 로 변환하여 표현한다.

In [ ]:
data_2gram = np.asarray(data_2gram)

다시 한 번 첫 번째 트레이스의 인코딩을 살펴보자.

In [ ]:
project_nth(sepsis_log, 0)

In [ ]:
print(data_2gram[0])

### Encoding as Bag of Words / Multiset of Events

또 다른 선택지는 자연어처리(NLP)에서 사용되는 [bag-of-words model](https://en.wikipedia.org/wiki/Bag-of-words_model) 모델로 알려진 인코딩 방식을 활용하는 것이다. 이는 원-핫 인코딩된 이벤트들을 멀티셋(multiset) 형태로 구성하여, 각 activity가 얼마나 자주 발생했는지를 빈도로 반영하는 방식이다.

즉, 단순히 활동이 존재하는지 여부만을 표현하는 것이 아니라, 각 활동의 발생 빈도까지 특징 벡터에 담게 된다.

이 인코딩은 PM4Py에서 기본적으로 제공되지는 않지만, Pandas와 Numpy를 사용하면 쉽게 계산할 수 있다.

가장 먼저 이벤트 로그를 데이터프레임 형태로 변환한다.

In [ ]:
sepsis_df = pm4py.convert_to_dataframe(sepsis_prefixes)
sepsis_df.head(25)

데이터를 그룹화한 뒤, 각 개별 activity에 해당하는 이벤트의 발생 횟수를 세어 bag-of-words 표현을 구축한다.

즉, 트레이스 단위로 이벤트를 묶은 후, 활동별 빈도를 계산하여 각 트레이스를 활동 발생 빈도 벡터로 변환하는 방식이다.

In [ ]:
# concept:name refers to the activity
# case:concept:name refers to the case identifier
sepsis_case_act = sepsis_df.loc[:,["case:concept:name", "concept:name"]]
sepsis_case_act

In [ ]:
# Count the occurrence of activities in a trace (no sorting to keep order of traces stable!)
sepsis_act_count = sepsis_case_act.groupby(["case:concept:name", "concept:name"], sort=False).size()
sepsis_act_count

각 트레이스마다 activity별 발생 횟수를 이미 집계했으므로, 이제 이를 텐서 형식으로 변환해야 한다. 즉, 행은 개별 케이스를, 열은 각 activity를, 값은 해당 케이스에서 activity가 발생한 횟수를 나타내도록 구성한다.

In [ ]:
sepsis_bag = np.asarray(sepsis_act_count.unstack(fill_value=0))
sepsis_bag

In [ ]:
sepsis_bag.shape

다시 한 번 첫 번째 트레이스의 인코딩을 살펴보자.

In [ ]:
project_nth(sepsis_log, 0)
print(sepsis_bag[0])

In [ ]:
project_nth(sepsis_log, 1)
print(sepsis_bag[1])

이 방식은 이미 훨씬 더 풍부한 정보를 제공해 준다.

## Prediction

이제 이 정보를 바탕으로 기본적인 예측 모델을 만들어 본다. 본 예제에서는 이벤트 `Return ER`이 발생했는지 여부를 이진 결과(binary outcome) 로 예측하는 것을 목표로 한다.

**주의사항: 여기서 기본적(basic) 이라는 것은 모델과 인코딩 방식이 높은 품질을 보장하지 않는다는 것을 의미한다. 또한 이번에 선택한 접두(prefix) 인코딩만으로는 실제로 유용한 예측이 가능하지 않을 수도 있다. 따라서 아래의 코드는 단지 예시와 출발점으로만 생각하시기 바란다.**

### Data Preparation

#### Target Variable

타겟 변수의 분포를 살펴보자.

In [ ]:
np.unique(sepsis_returns, return_counts=True)

In [ ]:
# For future processing we need 0 and 1 instead of True and False
sepsis_returns = np.asarray(sepsis_returns).astype(int)
sepsis_returns.shape

#### Data Scaling & Loading

이러한 전처리 과정은 예측 모델의 성능을 향상시키는 데 자주 도움이 된다.

**중요**: 반드시 테스트 셋을 포함한 상태에서 스케일링을 계산하지 않도록 주의해야 한다. 그렇지 않으면 데이터 누수(data leakage)가 발생할 위험이 있다. 다시 말해, 데이터셋의 어떤 속성을 활용하는 전처리를 수행하기 전에 반드시 테스트 셋을 분리해야 한다.

In [ ]:
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler

scaler_x = MinMaxScaler()
data_scaled = scaler_x.fit_transform(sepsis_bag)

scaler_y = FunctionTransformer() # for binary values scaling does not make sense at all but we keep it for symetry and apply the "NoOp" scaler
target_scaled = scaler_y.fit_transform(sepsis_returns.reshape(-1, 1))

### Model Definition

간단한 신경망을 정의하고 의도적으로 과적합(overfit) 을 시도해 본다. PyTorch를 사용해 기본적인 신경망(Neural Network)을 구축한다.

**주의사항: 다시 한 번 강조하지만, 아래 내용은 단순한 예시일 뿐이며 모델 권장안이 전혀 아니다.**

In [ ]:
class NeuralNetworkBinaryOutcome(nn.Module):
    def __init__(self):
        super(NeuralNetworkBinaryOutcome, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            torch.nn.Linear(x.shape[1], 64),
            nn.BatchNorm1d(num_features=64),
            nn.LeakyReLU(),
            torch.nn.Linear(64, 128),
            nn.BatchNorm1d(num_features=128),
            torch.nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

PyTorch에서 우리는 표준적인 학습 루프(training loop) 를 사용합니다.

(학습루프: Forward pass - Loss 계산 - Backward pass - Optimizer step - Gradient 초기화)

In [ ]:
def train(dataloader, model,
          loss_fn, measure_fn,
          optimizer, device, epochs):

    losses = []
    size = len(dataloader.dataset)

    loop = tqdm(range(epochs))

    for epoch in loop:

        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()

            # Compute prediction error
            pred = model(X)

            loss = loss_fn(pred, y)
            measure = measure_fn(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()

            losses.append([loss.item(), measure.item()])

        loop.set_description('Epoch {}/{}'.format(epoch + 1, epochs))
        loop.set_postfix(loss=loss.item(), measure=measure.item())

    return losses

그리고 다음 함수를 사용하여 모든 평가 결과를 얻을 수 있다.

In [ ]:
def evaluate_all(dataloader, model, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()

    result = []
    original = []

    with torch.no_grad():
        for X, y in tqdm(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)

            result.extend(pred.flatten().numpy())
            original.extend(y.flatten().numpy())

    return np.asarray(result), np.asarray(original)

### Training

PyTorch의 데이터 로딩 메커니즘을 사용하기 위해서는 먼저 데이터를 Dataset과 DataLoader 형태로 준비해야 한다.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# We need float32 data
x = torch.from_numpy(data_scaled.astype('float32'))
y = torch.from_numpy(target_scaled.astype('float32'))

# Always check the shapes
print(x.shape)
print(y.shape)

ds = TensorDataset(x, y)
train_dataloader = DataLoader(ds, batch_size=64, shuffle=True)

우리의 data loader로 부터 임의의 샘플에 대해 확인한다.

In [ ]:
inputs, classes = next(iter(train_dataloader))
print(inputs[0])
print(classes[0])

모델은 교차 엔트로피(cross entropy) 를 손실 함수로 사용하여 학습한다. 그리고 성능을 보고할 때는 해석하기 더 쉬운 지표인 정확도(accuracy) 를 사용한다.

In [ ]:
## if you want ot use a GPU you need to tweak the requirements.txt to include the GPU-enabled PyTorch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

# fix a seed to get reproducible results
torch.manual_seed(42)

model = NeuralNetworkBinaryOutcome().to(device)
print(model)

def get_accuracy(y_prob, y_true):
    y_true = y_true.flatten()
    y_prob = y_prob.flatten()
    assert y_true.ndim == 1 and y_true.size() == y_prob.size()
    y_prob = y_prob > 0.5
    return (y_true == y_prob).sum() / y_true.size(0)
measure_fn = get_accuracy

results = train(train_dataloader, model,
                nn.BCELoss(), # crossentropy for binary target
                get_accuracy,
                torch.optim.Adam(model.parameters()),
                device, 100)

In [ ]:
%matplotlib inline

results_data = pd.DataFrame(results)
results_data.columns = ['loss', 'measure']
ax = results_data.plot(subplots=True);

In [ ]:
print("Accuracy: " + str(results[len(results)-1][1]))

true_returns = np.unique(sepsis_returns, return_counts=True)[1][0]
true_not_returns = np.unique(sepsis_returns, return_counts=True)[1][1]

print("Accuracy (never returns)" + str(true_returns / len(sepsis_returns)))
print("Accuracy (always returns)" + str(true_not_returns / len(sepsis_returns)))

## Brief Evaluation

단순히 환자는 재방문하지 않는다 라고만 예측하는 것보다는 조금 나아졌다. 그러나 훈련 셋에서의 정확도조차 여전히 크게 변동하며, 학습된 epoch에 따른 손실 값과 정확도의 변동 추이도 그다지 안정적이지 않다. 따라서, 이제는 개별 예측 결과를 살펴보고, 실제 정답(ground truth)에 따라 예측 점수가 어떻게 분포하는지 확인해 본다.

In [ ]:
test_dataloader = DataLoader(ds, batch_size=256, shuffle=False)
result, original = evaluate_all(test_dataloader, model, device)

In [ ]:
pd_pos = pd.DataFrame({'Returns': result[original == 1]})
pd_neg = pd.DataFrame({'Does not return': result[original == 0]})
pd.concat([pd_pos, pd_neg],axis=1).boxplot().set_ylabel('Score')

일정한 분리(separation)는 보이지만, 실제로 환자의 재방문을 식별하는 데 이 예측 모델을 사용한다면 거짓 양성(false positive) 이 많이 발생할 가능성이 높아 보인다.

**이제는 분류(classification) 과제에서 일반적으로 사용하는 다양한 평가 지표들을 계산해야 한다. 예를 들어, 재현율(Recall), 정밀도(Precision), 혼동 행렬(Confusion Matrix), ROC 곡선과 AUC (Area Under the Curve) 등을 포함한 여러 방법으로 예측 모델을 평가할 수 있다.**

이와 관련하여 데이터 분포를 살펴보자.

In [ ]:
# count the unique vectors
dist_bags = np.unique(sepsis_bag, return_counts=True, axis=0)

# sort them with numpy
unique_vectors = dist_bags[0][np.argsort(-dist_bags[1])]
count_vectors = dist_bags[1][np.argsort(-dist_bags[1])]

pd.DataFrame({'Occurrence of unique sample vectors': count_vectors}).boxplot().set_ylabel('Frequency')

많은 트레이스가 완전히 동일한 샘플로 귀결된다. 이제 175개 이상의 트레이스를 대표하는 가장 흔한 샘플에 대해, 그 샘플의 “재방문 상태(return status)” 가 무엇인지 확인해 본다.

In [ ]:
# most frequently used vector
unique_vectors[0]

In [ ]:
# find the sample indicies for this vector
sample_indicies = np.where((sepsis_bag == unique_vectors[0]).all(axis=1))
sample_durations = target_scaled[sample_indicies]

In [ ]:
np.unique(sample_durations, return_counts=True)

분명히 알 수 있는 것은 추가적인 정보가 없다면 동일한 feature 값에 대해서는 예측 모델이 구분을 학습할 수 없다는 점이다. 더 많은 예시를 살펴볼 수는 있겠지만, Sepsis 이벤트 로그에서 bag-of-words / 이벤트 멀티셋 기반 모델만으로는 환자가 재방문할지 여부를 신뢰성 있게 예측하기 어렵다는 결론에 이른다.

이번 예시는 단지 이벤트 로그에 포함된 이벤트들을 기반으로, 프로세스의 이진 특성(binary process characteristic) 을 예측하기 위해 예측 모델을 어떻게 활용할 수 있는지를 보여주기 위한 예시일 뿐이다.